# Meteo data reading

Data source: https://datacatalog.regione.emilia-romagna.it/catalogCTA/dataset/meteo-dati-osservati2

In [1]:
import pandas as pd, ast, os, plotly.express as px, numpy as np, geopandas as gpd, re
from scipy import stats

## Reading a single year

In [2]:
%%time

# leggo il file csv
read_2021 = pd.read_csv('Storico meteo/unito_2021.csv')

CPU times: total: 23.4 s
Wall time: 1min


In [3]:
read_2021

,Unnamed: 0,version,network,ident,lon,lat,date,data
0,0,0.1,agrmet,NaN,1095670,4470214,2021-02-01 00:00:00+00:00,"[{'vars': {'B01019': {'v': 'Albareto'}, 'B0119..."
1,1,0.1,agrmet,NaN,1095670,4470214,2021-02-01 00:15:00+00:00,"[{'vars': {'B01019': {'v': 'Albareto'}, 'B0119..."
2,2,0.1,agrmet,NaN,1095670,4470214,2021-02-01 00:30:00+00:00,"[{'vars': {'B01019': {'v': 'Albareto'}, 'B0119..."
3,3,0.1,agrmet,NaN,1095670,4470214,2021-02-01 00:45:00+00:00,"[{'vars': {'B01019': {'v': 'Albareto'}, 'B0119..."
4,4,0.1,agrmet,NaN,1095670,4470214,2021-02-01 01:00:00+00:00,"[{'vars': {'B01019': {'v': 'Albareto'}, 'B0119..."
...,...,...,...,...,...,...,...,...
16922941,16922941,0.1,spdsra,NaN,1219660,4395814,2021-01-31 18:00:00+00:00,[{'vars': {'B01019': {'v': 'Mercato Saraceno'}...
16922942,16922942,0.1,spdsra,NaN,1219660,4395814,2021-01-31 19:00:00+00:00,[{'vars': {'B01019': {'v': 'Mercato Saraceno'}...
16922943,16922943,0.1,spdsra,NaN,1219660,4395814,2021-01-31 20:00:00+00:00,[{'vars': {'B01019': {'v': 'Mercato Saraceno'}...
16922944,16922944,0.1,spdsra,NaN,1219660,4395814,2021-01-31 21:00:00+00:00,[{'vars': {'B01019': {'v': 'Mercato Saraceno'}...


In [4]:
%%time

# Convert the 'data' column to a string
read_2021['data'] = read_2021['data'].astype(str)

# Combine all 'data' strings into one large string
combined_data = ' '.join(read_2021['data'])

# Use regular expressions to find all codes in the format 'B04006'
found_codes = re.findall(r"'B\d{5}':", combined_data)

# Remove duplicates from the list of found codes
unique_codes = list(set(found_codes))

CPU times: total: 26 s
Wall time: 1min 10s


In [5]:
# Remove unwanted characters from each code
cleaned_codes = [code.replace("'", "").replace(",", "").replace(":", "") for code in unique_codes]

In [6]:
len(cleaned_codes)

31

In [7]:
# Check if the file "Storico meteo/Processed_data/meteo_codes.csv" exists
if os.path.exists("Storico meteo/Processed_data/meteo_codes.csv"):
    print("File 'Storico meteo/Processed_data/meteo_codes.csv' already exists.")
else:
    # Specify the URL from which you want to download the table
    url = 'https://arpa-simc.github.io/dballe/general_ref/btable.html'  # Replace with your actual URL

    # Use pd.read_html() to download the table from the web page
    # This method will return a list of DataFrames, one for each table found on the page
    tables = pd.read_html(url)

    # Select the table you're interested in from the list (there may be more than one)
    # Let's assume you want the first table, so select tables[0]
    your_table = tables[0]
    
    your_table.to_csv("Storico meteo/Processed_data/meteo_codes.csv", index=False)

File 'Storico meteo/Processed_data/meteo_codes.csv' already exists.


In [8]:
# Specify the file path
file_path = "Storico meteo/Processed_data/meteo_codes.csv"

# Read the CSV file into a DataFrame
your_table = pd.read_csv(file_path)

In [9]:
# Filter rows based on the cleaned codes
your_table_filtered = your_table[your_table['Code'].isin(cleaned_codes)].reset_index(drop=True)

In [10]:
your_table_filtered

,Code,Description,Unit,Format
0,B01001,WMO BLOCK NUMBER,Numeric,3 digits
1,B01002,WMO STATION NUMBER,Numeric,4 digits
2,B01019,LONG STATION OR SITE NAME,CCITTIA5,32 digits
3,B01194,Report mnemonic,CCITTIA5,16 digits
4,B04001,YEAR,YEAR,4 digits
5,B04002,MONTH,MONTH,2 digits
6,B04003,DAY,DAY,2 digits
7,B04004,HOUR,HOUR,2 digits
8,B04005,MINUTE,MINUTE,2 digits
9,B04006,SECOND,SECOND,2 digits


In [11]:
# Specify the file path
file_path = "Storico meteo/Processed_data/meteo_codes_filtered.csv"

your_table_filtered.to_csv(file_path, index=False)

In [2]:
def extract_data_and_create_dataframe(df, variables_of_interest):
    # Filter the DataFrame based on the first variable of interest
    mask = df['data'].str.contains('|'.join(variables_of_interest))

    # Filtrare il DataFrame utilizzando la maschera booleana
    df_filtered = df[mask].reset_index()

    # Inizializza una lista vuota per memorizzare i timerange_key unici
    timerange_keys_list = []

    # Itera attraverso il DataFrame df
    for index, row in df_filtered.iterrows():
        # Estrai il valore dalla colonna 'data'
        data_str = row['data']

        # Analizza la stringa JSON
        json_data = ast.literal_eval(data_str)

        # Inizializza timerange a None
        timerange = None

        # Itera attraverso gli oggetti JSON
        for obj in json_data[1:]:
            obj_str = str(obj)
            if 'timerange' in obj_str and 'B13011' in obj_str:
                # Crea il timerange_key
                timerange = obj['timerange']
                timerange_key = f'B13011_{timerange[0]}_{timerange[-1]}'

                # Aggiungi il timerange_key alla lista se non è già presente
                if timerange_key not in timerange_keys_list:
                    timerange_keys_list.append(timerange_key)

    name_list = []

    for i in range(len(df_filtered)):
        date = df_filtered['date'][i]
        get_name = df_filtered['data'][i]
        json_data = ast.literal_eval(get_name)

        values_dict = {}  # Inizializza un dizionario per memorizzare i valori per B13011
        values_dict_timerange = {}

        for i in range(0, len(json_data)):
            for j, variable in enumerate(variables_of_interest):
                try:
                    if variable == 'B13011':
                        timerange = json_data[i].get('timerange')
                        if timerange is not None:
                            timerange_key = f'{variable}_{timerange[0]}_{timerange[-1]}'  # Crea una colonna separata per l'ultimo timerange
                            value = json_data[i]['vars'][variable]['v']
                            values_dict_timerange[timerange_key] = value

                    else:
                        timerange = json_data[i].get('timerange')
                        if timerange is not None:
                            if str(timerange[0]) == '254':
                                value = json_data[i]['vars'][variable]['v']
                                if variable in variables_of_interest:
                                    values_dict[variable] = value
                        else:
                            continue

                except KeyError:
                    continue

        # Aggiungi i valori del dizionario alle colonne della lista dei valori
        values = [values_dict.get(variable, np.nan) for variable in variables_of_interest if variable != 'B13011']
        values_keys = [values_dict_timerange.get(variable, np.nan) for variable in timerange_keys_list]

        name_list.append([json_data[0]['vars']['B01019']['v'], json_data[0]['vars']['B05001']['v'], json_data[0]['vars']['B06001']['v'],
                          json_data[0]['vars']['B07030']['v'], json_data[0]['vars'].get('B07031', {}).get('v', np.nan)] + values + values_keys + [date])


    total_columns = [variable for variable in variables_of_interest if variable != 'B13011'] + timerange_keys_list

    df_station = pd.DataFrame(name_list)
    df_station.columns = ['name', 'lon', 'lat', 'h_sea', 'h_bar'] + total_columns + ['date']

    # Elimina le righe in cui 'Colonna1' e 'Colonna2' contengono tutti NaN
    df_station = df_station.dropna(subset=total_columns, how='all').reset_index(drop=True)

    return df_station

In [ ]:
%%time

# Utilizzo della funzione con un DataFrame generico
variables_of_interest = ['B11001','B11002', 'B11041', 'B11043', 'B12101', 'B13003', 'B13011']
variable_to_mode = ['B11001', 'B11041', 'B11043', 'B12101', 'B13003', 'B13011']
df_station_2021 = extract_data_and_create_dataframe(read_2021, variables_of_interest, variable_to_mode)

In [ ]:
df_station_2021['name'].unique()

In [ ]:
single_station = df_station_2021[df_station_2021['name'] == 'Cesenatico porto']
single_station = single_station.sort_values(by=['date'])

In [ ]:
single_station

In [ ]:
df_station_2021.to_csv("Storico meteo/Processed_data/all_variables_2021_modify.csv", index=False)

## Reading all the years available

In [3]:
%%time

# Lista per memorizzare i DataFrame da ciascun file CSV
dfs = []
dir_path = 'Storico meteo'
variables_of_interest = ['B11001','B11002']
variable_to_mode = ['B11001']

# Elencare tutti i file CSV nella cartella
for filename in os.listdir('Storico meteo'):
    if filename.endswith(".csv"):
        # Print the filename being processed
        print("Processing file:", filename)
        file_path = os.path.join(dir_path, filename)
        df = pd.read_csv(file_path)
        df_station = extract_data_and_create_dataframe(df, variables_of_interest, variable_to_mode)
        dfs.append(df_station)

# Concatenare tutti i DataFrame in uno solo
final_df = pd.concat(dfs, ignore_index=True)

Processing file: unito_2012.csv
Processing file: unito_2013.csv
Processing file: unito_2014.csv
Processing file: unito_2015.csv
Processing file: unito_2016.csv
Processing file: unito_2017.csv
Processing file: unito_2018.csv
Processing file: unito_2019.csv
Processing file: unito_2020.csv
Processing file: unito_2021.csv
CPU times: total: 42min 47s
Wall time: 1h 2min 50s


In [4]:
final_df.to_csv("Storico meteo/Processed_data/wind_variables_old_code.csv", index=False)